In [ ]:
%%sh
pygmentize sklearn-boston-housing-generic.py

In [ ]:
%%sh
cat Dockerfile-generic

In [1]:
%%sh
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:estimator -f Dockerfile-generic .
export IMAGE_ID=`docker images -q sklearn-custom:estimator`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator


Step 1/4 : FROM python:3.7
 ---> 072863679c31
Step 2/4 : RUN pip3 install --no-cache scikit-learn numpy pandas joblib
 ---> Using cache
 ---> b0cc35081cdb
Step 3/4 : COPY sklearn-boston-housing-generic.py /usr/bin/train
 ---> Using cache
 ---> 6e7db1d65598
Step 4/4 : RUN chmod 755 /usr/bin/train
 ---> Using cache
 ---> bd407936dab6
Successfully built bd407936dab6
Successfully tagged sklearn-custom:estimator
Login Succeeded
The push refers to repository [613904931467.dkr.ecr.eu-west-1.amazonaws.com/sklearn-custom]
76f77313f364: Preparing
04ee4b880642: Preparing
f58507b91a56: Preparing
3dcbd9a42407: Preparing
35dd3ebed11d: Preparing
f61899846314: Preparing
697765a85531: Preparing
8c39f7b1a31a: Preparing
88cfc2fcd059: Preparing
760e8d95cf58: Preparing
7cc1c2d7e744: Preparing
8c02234b8605: Preparing
760e8d95cf58: Waiting
697765a85531: Waiting
7cc1c2d7e744: Waiting
8c02234b8605: Waiting
8c39f7b1a31a: Waiting
88cfc2fcd059: Waiting
f61899846314: Waiting
35dd3ebed11d: Layer already exists
f58

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [13]:
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()

sk = Estimator(
             image_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:estimator',
             role=role,
             train_instance_count=1, 
             train_instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1,
                  'random-state': 123
             }
)

sk.fit({'training':training})

Creating tmpibm268ox_algo-1-6krfi_1 ... 
Attaching to tmpibm268ox_algo-1-6krfi_12mdone
algo-1-6krfi_1  | {'normalize': 'True', 'test-size': '0.1', 'random-state': '123'}
algo-1-6krfi_1  | Mean squared error: 41.02
algo-1-6krfi_1  | Coefficient of determination: 0.64
tmpibm268ox_algo-1-6krfi_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


In [ ]:
%%sh
tar tvfz model.tar.gz